In [1]:
import time

import numpy as np
from numpy import random

from pathlib import Path

import shutil

import cv2
from IPython import display

import torch
from torch import nn
import torch.nn.functional as F

import onnx
import onnxruntime

from utils.image_utils import cv2_imshow
from models import yolov5_onnx

from detect import read_image, load_names, overlay_boxes

In [2]:
import os

os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"]="0"

device = torch.device('cuda')

## Model Definition and Initialization

In [3]:
model = yolov5_onnx(pretrained=True, min_size=320, max_size=416, score_thresh=0.5)

model = model.eval()
model = model.to(device)

### Load images to infer

In [4]:
path0 = './test/assets/bus.jpg'
path1 = './test/assets/zidane.jpg'

img_test0 = read_image(path0, is_half=False)
img_test0 = img_test0.to(device)

img_test1 = read_image(path1, is_half=False)
img_test1 = img_test1.to(device)

images = [img_test0, img_test1]

### Inference in `pytorch` backend

In [5]:
with torch.no_grad():
    model_out = model(images)

In [6]:
model_out[0]['boxes']

tensor([[ 48.4231, 401.9458, 237.0045, 897.8144],
        [215.4538, 407.8977, 344.6994, 857.3773],
        [ 13.1457, 225.1691, 801.7442, 736.7350],
        [675.6570, 409.5675, 812.7283, 868.2495]], device='cuda:0')

In [7]:
model_out[0]['scores']

tensor([0.8941, 0.8636, 0.8621, 0.7490], device='cuda:0')

In [8]:
model_out[0]['labels']

tensor([0, 0, 5, 0], device='cuda:0')

## Export to ONNX model

In [9]:
# TorchScript export
print(f'Starting ONNX export with onnx {onnx.__version__}, onnxruntime {onnxruntime.__version__}...')
export_onnx_name = './checkpoints/yolov5/yolov5s.onnx'

Starting ONNX export with onnx 1.8.0, onnxruntime 1.5.2...


In [10]:
from torchvision.ops._register_onnx_ops import _onnx_opset_version

In [11]:
# Export to ONNX model
torch.onnx.export(
    model,
    (images,),
    export_onnx_name,
    do_constant_folding=True,
    opset_version=_onnx_opset_version,
    dynamic_axes={"images_tensors": [0, 1, 2], "outputs": [0, 1, 2]}, 
    input_names=["images_tensors"],
    output_names=["outputs"],
)

/usr/local/lib/python3.6/dist-packages/torch/onnx/utils.py:1112: UserWarning: No names were found for specified dynamic axes of provided input.Automatically generated names will be applied to each dynamic axes of input images_tensors
  'Automatically generated names will be applied to each dynamic axes of input {}'.format(key))
/usr/local/lib/python3.6/dist-packages/torch/onnx/utils.py:1112: UserWarning: No names were found for specified dynamic axes of provided input.Automatically generated names will be applied to each dynamic axes of input outputs
  'Automatically generated names will be applied to each dynamic axes of input {}'.format(key))
/usr/local/lib/python3.6/dist-packages/torch/nn/functional.py:3123: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  dtype=torch.float32)).float())) for i in range(dim)]
/mnt/yolov5-rt-stack/models/

## Inference on `ONNXRuntime` Backend

In [12]:
images, _ = torch.jit._flatten(images)
outputs, _ = torch.jit._flatten(model_out)

In [13]:
def to_numpy(tensor):
    if tensor.requires_grad:
        return tensor.detach().cpu().numpy()
    else:
        return tensor.cpu().numpy()

In [14]:
inputs = list(map(to_numpy, images))
outputs = list(map(to_numpy, outputs))

In [15]:
ort_session = onnxruntime.InferenceSession(export_onnx_name)
# compute onnxruntime output prediction
ort_inputs = dict((ort_session.get_inputs()[i].name, inpt) for i, inpt in enumerate(inputs))
ort_outs = ort_session.run(None, ort_inputs)

In [16]:
for i in range(0, len(outputs)):
    torch.testing.assert_allclose(outputs[i], ort_outs[i], rtol=1e-03, atol=1e-05)

print("Exported model has been tested with ONNXRuntime, and the result looks good!")

Exported model has been tested with ONNXRuntime, and the result looks good!
